In [1]:
#04/2023

import pandas as pd
import numpy as np
import random
import math
import matplotlib.pyplot as plt
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, mean_squared_error

# Weighted Sum

In [2]:
def weightedSum(bias, x_input, w_weights):
    weighted_sum = 0
    
    for x, w in zip(x_input, w_weights):
        weighted_sum += x * w
        
    weighted_sum += bias
    
    return weighted_sum

# Activation functions

## Step

In [3]:
def step(weighted_sum):
    
    if type(weighted_sum) != float:
        weighted_sum = float(weighted_sum[0])
        
    threshold = 0
    
    if weighted_sum > threshold:
        return 1
    else:
        return 0
    
    return pred

## Sigmoid

In [4]:
def sigmoid(weighted_sum):
    
    pred = 1 / (1 + np.exp(- weighted_sum))
    
    return pred

## Linear

In [5]:
def linear(weighted_sum):
    return weighted_sum

# Gradient Descent


* Updating weights
* Learning rate
    * It's a very small floating point scalar, for example: 0.1, 0.01, 0.001
    * ensures a gradual weight update
* Bias

In [6]:
def gradient(w_weights, bias, learning_rate, x_train, y_train, pred, activation):
    
    new_weights = []
    bias += learning_rate * (y_train - pred)
    
    
    if activation == 'sigmoid':
            for x, w in zip(x_train, w_weights):
                new_w = w + learning_rate * (y_train - pred) * x * (np.exp(-x)/(1+np.exp(-x))**2) # derivada da sig
                new_weights.append(new_w)
        
    else:
            for x, w in zip(x_train, w_weights):
                new_w = w + learning_rate * (y_train - pred) * x
                new_weights.append(new_w) 
    
    return new_weights, bias

# Learning rate update

In [7]:
def stocastic_learning_rate(learning_rate, j):
    learning_rate = learning_rate / (1 + j)
    
    return learning_rate

def fixed_learning_rate(first_learning_rate, j):
    return first_learning_rate

def adaptative_learning_rate(first_learning_rate, j):
    learning_rate = first_learning_rate / (1 + j)
    
    return learning_rate

# Perceptron

In [8]:
def perceptron(x_train, x_test, y_train, y_test, learning_rate, epochs, activation, learning):
    
    first_learning_rate = learning_rate
    
    # Activation selection
    activation_functions = {
        'step': step,
        'sigmoid': sigmoid,
        'linear': linear
    }
    
    activation_function = activation_functions.get(activation)
    
    learning_rate_func = {
        'stocastic': stocastic_learning_rate,
        'fixed': fixed_learning_rate,
        'adaptative': adaptative_learning_rate
    }
    
    learning_function = learning_rate_func.get(learning)
    
    # Inicial values (weight, bias)
    w_weights = [random.gauss(0, 0.5) for _ in range(x_train.shape[1])] # inicial random weights
    bias = random.gauss(0, 0.5)
    weighted_sum = 0
    
    # Prediciont list
    y_preds = []

    # train step
    j = 0
    while (j != epochs):
        for row in range(len(x_train)):
            feature = x_train.loc[row]
            target = y_train.loc[row]

            weighted_sum = weightedSum(bias, feature, w_weights)
            pred = activation_function(weighted_sum)
            w_weights, bias = gradient(w_weights, bias, learning_rate, feature, target, pred, activation)
            
        if learning != 'stocastic':
            learning_rate = learning_function(first_learning_rate, j)
        else:
            learning_rate = learning_function(learning_rate, j)
        j += 1
    
    # test step
    for row in range(len(x_test)):
        feature = x_test.loc[row]
        weighted_sum = weightedSum(bias, feature, w_weights)
        pred = activation_function(weighted_sum)
        
        if type(pred) != int:
            pred = float(pred[0])
        
        y_preds.append(pred)

    return y_preds, w_weights

### Normalize dataframe

In [9]:
def normalize(DataFrame):
    df = DataFrame

    min = df.values.min()
    max = df.values.max()

    df_minmax = (df-min)/(max-min)
    return df_minmax,min,max

# Iris

In [10]:
# Loading dataset
iris = datasets.load_iris()

# to Pandas DataFrame
df_iris = pd.DataFrame(iris.data, columns=iris.feature_names)
df_iris, min, max = normalize(df_iris) # normalizing values
df_iris['target'] = pd.Series(iris.target)



# Filtering target "setosa" and "versicolor"
df_filtered = df_iris[df_iris['target'].isin([0, 1])]

# Reset index
df_filtered.reset_index(drop=True, inplace=True)

# Spliting features and targets
X = df_filtered.drop('target', axis=1)
y = df_filtered['target']

# Train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reset_index().drop('index', axis=1)
X_test = X_test.reset_index().drop('index', axis=1)
y_train = y_train.reset_index().drop('index', axis=1)
y_test = y_test.reset_index().drop('index', axis=1)

# Classification Iris

### 1. Sigmoid activation function + fixed learning rate

In [11]:
perceptron_iris, pesos = perceptron(X_train, X_test, y_train, y_test, 0.5, 7, 'sigmoid', 'fixed')

perceptron_iris # predictions

[0.8762542360145814,
 0.7963940146092213,
 0.8624197502892146,
 0.4494079878068249,
 0.5053323996069026,
 0.45073838122967835,
 0.38635019242856145,
 0.7701222795246678,
 0.44854730268360654,
 0.4375313529885993,
 0.47966603570017013,
 0.4646936200449377,
 0.8398238020582807,
 0.4301173915476938,
 0.8187648621207458,
 0.43524356424847344,
 0.8557647954802897,
 0.8744529387370351,
 0.436077562070868,
 0.46651803033552824]

In [12]:
binary_preds = [1 if pred > 0.5 else 0 for pred in perceptron_iris]

In [13]:
# Acuracy score
acc1 = accuracy_score(y_test, binary_preds)
acc1

0.95

### 1.2 Sigmoid activation function + stocastic learning rate

In [14]:
perceptron_iris, pesos = perceptron(X_train, X_test, y_train, y_test, 0.5, 7, 'sigmoid', 'stocastic')

perceptron_iris # predictions

[0.7570707541937093,
 0.6691838948467352,
 0.7401192016939068,
 0.4227140874732414,
 0.4732108489532993,
 0.4311957165555479,
 0.3882695230462265,
 0.6499683953521704,
 0.4315495726845489,
 0.42244688814825204,
 0.4510501942760363,
 0.4400755662681759,
 0.7235974754857253,
 0.424234122831364,
 0.7014063910219889,
 0.423207699796644,
 0.7316445472450307,
 0.7505280014581597,
 0.4197222479495439,
 0.43323677389894233]

In [15]:
binary_preds = [1 if pred > 0.5 else 0 for pred in perceptron_iris]

In [16]:
# Acuracy score
acc1 = accuracy_score(y_test, binary_preds)
acc1

1.0

### 1.3 Sigmoid activation function + adaptative learning rate

In [17]:
perceptron_iris, pesos = perceptron(X_train, X_test, y_train, y_test, 0.5, 7, 'sigmoid', 'adaptative')

perceptron_iris # predictions

[0.7713502645274262,
 0.6743456492284148,
 0.7585985161507642,
 0.41679983974168555,
 0.4799472297918893,
 0.43332618346162366,
 0.3836038096074757,
 0.654741731925407,
 0.4420178140482622,
 0.42559837981521303,
 0.46884419378372416,
 0.43369683564634026,
 0.739088261883106,
 0.4410537570414528,
 0.707085285448209,
 0.4253050015914991,
 0.758240407518683,
 0.777331543252276,
 0.4120451208327827,
 0.4429392459643644]

In [18]:
print(pesos)

[target    0.505919
dtype: float64, target    0.412304
dtype: float64, target    2.872393
dtype: float64, target    0.764127
dtype: float64]


In [19]:
binary_preds = [1 if pred > 0.5 else 0 for pred in perceptron_iris]

In [20]:
# Acuracy score
acc1 = accuracy_score(y_test, binary_preds)
acc1

1.0

### 2. Step activation function + fixed learning rate

In [21]:
perceptron_iris, pesos = perceptron(X_train, X_test, y_train, y_test, 0.5, 7, 'step', 'fixed')

perceptron_iris # predictions

[1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0]

In [22]:
# Acurary score
acc2 = accuracy_score(y_test, perceptron_iris)
acc2

1.0

### 2.2 Step activation function + stocastic learning rate

In [23]:
perceptron_iris, pesos = perceptron(X_train, X_test, y_train, y_test, 0.5, 7, 'step', 'stocastic')

perceptron_iris # predictions

[1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0]

In [24]:
# Acurary score
acc2 = accuracy_score(y_test, perceptron_iris)
acc2

1.0

### 2.3 Step activation function + adaptative learning rate

In [25]:
perceptron_iris, pesos = perceptron(X_train, X_test, y_train, y_test, 0.5, 7, 'step', 'adaptative')

perceptron_iris # predictions

[1, 1, 1, 0, 0, 0, 0, 1, 0, 0, 0, 0, 1, 0, 1, 0, 1, 1, 0, 0]

In [26]:
# Acurary score
acc2 = accuracy_score(y_test, perceptron_iris)
acc2

1.0

# Regression

In [27]:
# Creating a regression dataset
x1 = np.random.rand(200)
x2 = np.random.rand(200)
x3 = np.random.rand(200)

x4 = 0.3 * x1 + 0.2 * x2 + 0.8 * x3 + 0.05

regression_data = pd.DataFrame({
    'X1': x1,
    'X2': x2,
    'X3': x3,
    'X4': x4
})

regression_data

,X1,X2,X3,X4
0,0.662304,0.307501,0.725569,0.890647
1,0.806700,0.033413,0.228427,0.481434
2,0.493973,0.999878,0.207023,0.563786
3,0.935626,0.905713,0.903737,1.234820
4,0.885396,0.621128,0.418826,0.774905
...,...,...,...,...
195,0.402820,0.322202,0.649471,0.754864
196,0.613381,0.281637,0.165011,0.422351
197,0.796132,0.726993,0.513465,0.845010
198,0.312061,0.757657,0.814357,0.946635


In [28]:
# Features and target
X = regression_data.drop(columns=['X4'])
y = regression_data['X4'] # target

# Train-test
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

X_train = X_train.reset_index().drop('index', axis=1)
X_test = X_test.reset_index().drop('index', axis=1)
y_train = y_train.reset_index().drop('index', axis=1)
y_test = y_test.reset_index().drop('index', axis=1)

### 1.1 Linear activation function + fixed learning rate

In [29]:
perceptron_regression, pesos = perceptron(X_train, X_test, y_train, y_test, 0.2, 5, 'linear', 'fixed')

perceptron_regression # predictions

[0.6031479082487711,
 0.8155606016761254,
 0.5315027573987596,
 0.5723018947026974,
 0.8059761014105574,
 0.5508769095127762,
 0.5948741707798937,
 0.9178152178535508,
 0.5022666987593727,
 0.9817434168828013,
 1.07999923977206,
 1.0131258760883104,
 0.48464710850138676,
 1.0160666274474952,
 1.0746055718398622,
 0.50123675492926,
 0.8511485310308577,
 1.169612844993042,
 0.7353305168971891,
 0.40705613383535655,
 0.7938690675201958,
 0.345006192871556,
 1.1031433887792113,
 0.9720840428234793,
 0.3384466173831127,
 0.6470740742797297,
 0.7398995575338799,
 0.40010343873150145,
 0.9925432683451009,
 0.5280293262674991,
 0.9977660436509015,
 0.9993840286847824,
 1.0824553383355047,
 0.9952909036366069,
 0.8859607544650203,
 0.7363714273109185,
 0.2672910906237932,
 0.747307672564669,
 0.5644278978547503,
 0.8338345805038847]

In [30]:
# MSE
mean_squared_error(y_test, perceptron_regression)

6.523604388822274e-09

In [31]:
# Perceptron weights 
pesos

[X4    0.300175
 dtype: float64,
 X4    0.200163
 dtype: float64,
 X4    0.800199
 dtype: float64]

### 1.2 Linear activation function + stocastic learning rate

In [32]:
perceptron_regression, pesos = perceptron(X_train, X_test, y_train, y_test, 0.2, 5, 'linear', 'stocastic')

perceptron_regression # predictions

[0.6026090597433769,
 0.8162489619242018,
 0.5322082292249745,
 0.5721581897002432,
 0.8054138444337596,
 0.5505455128030096,
 0.5955644402754389,
 0.9177417175933648,
 0.5020507570995729,
 0.9815289387501935,
 1.0805149930976847,
 1.0136258506171776,
 0.48452038250917095,
 1.0161779909634758,
 1.0745780054861649,
 0.5009754232329189,
 0.8508957575974344,
 1.170421204824163,
 0.7348195073651105,
 0.407297335254881,
 0.7934858366997006,
 0.34500083334732107,
 1.103976200535321,
 0.9723645376100719,
 0.3386411742993637,
 0.6469556048008815,
 0.7404905792272469,
 0.39941597347571545,
 0.9921687830088353,
 0.5284314444196219,
 0.9975423232306605,
 1.0001587755258132,
 1.0828643556957636,
 0.9958816273317028,
 0.8853513028113065,
 0.7357640367830628,
 0.2663917905634994,
 0.7472336283297122,
 0.5637976429127992,
 0.8343458773450435]

In [33]:
# MSE
mean_squared_error(y_test, perceptron_regression)

2.9822742750525894e-07

In [34]:
# Perceptron weights 
pesos

[X4    0.300887
 dtype: float64,
 X4    0.201789
 dtype: float64,
 X4    0.800607
 dtype: float64]

### 1.3 Linear activation function + adaptative learning rate

In [35]:
perceptron_regression, pesos = perceptron(X_train, X_test, y_train, y_test, 0.2, 5, 'linear', 'adaptative')

perceptron_regression # predictions

[0.603521645909341,
 0.8150352382623122,
 0.5311972937858755,
 0.5724878761734161,
 0.8059554081078238,
 0.5509931980774208,
 0.5945601679880507,
 0.9174660145435529,
 0.5024579891449368,
 0.981405874421992,
 1.0793000455266297,
 1.0125102393113943,
 0.48477620691035794,
 1.0156527296832851,
 1.0740140188283276,
 0.5014179514302138,
 0.8510045976702384,
 1.168585463851911,
 0.7354911893056754,
 0.40725979392554584,
 0.7938381149304509,
 0.34533267168544324,
 1.1022574651678299,
 0.9717211947353749,
 0.33876112950310644,
 0.6471498748956415,
 0.7394495481771827,
 0.4006914575047122,
 0.9922502236045075,
 0.5278586268674931,
 0.9973992894208118,
 0.9985890090905711,
 1.0817882258835743,
 0.9947076130964446,
 0.8859714730684158,
 0.7365513597955555,
 0.2681035449009419,
 0.7471547294573054,
 0.5647396726916004,
 0.8334824300820303]

In [36]:
# MSE
mean_squared_error(y_test, perceptron_regression)

1.301363547908476e-07

In [37]:
# Perceptron weights 
pesos

[X4    0.299276
 dtype: float64,
 X4    0.199235
 dtype: float64,
 X4    0.799143
 dtype: float64]